# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%matplotlib inline
import pickle
from os import listdir, path
import numpy as np
import matplotlib.pyplot as plt # Expliziter Import für Plotting-Funktionen
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data_converted' 

In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
# In der Zelle, wo load_pose_data(i) definiert wird (in deinem Python 3 Notebook):
import pickle # Stelle sicher, dass es importiert ist
from os import path # Stelle sicher, dass es importiert ist
# numpy as np sollte auch schon importiert sein

def load_pose_data(i):
    '''load pose data from file'''
    data_for_class = []
    target_for_class = []
    
    # ROBOT_POSE_DATA_DIR und classes sollten global im Notebook definiert sein
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i]) 
    
    try:
        with open(filename, 'rb') as f:
            # Mit protocol=2 gespeicherte Dateien sollten ohne explizites Encoding lesbar sein.
            # Falls es doch Probleme gibt, könnte man encoding='latin1' oder 'bytes' probieren,
            # aber es sollte nicht nötig sein.
            loaded_feature_vectors = pickle.load(f) 
        
        # Überprüfe, ob die geladenen Daten das erwartete Format haben (Liste von Listen)
        if not isinstance(loaded_feature_vectors, list):
            print(f"WARNUNG: Daten in {filename} sind keine Liste. Typ: {type(loaded_feature_vectors)}")
            return [], []
        if loaded_feature_vectors and not all(isinstance(item, (list, np.ndarray)) for item in loaded_feature_vectors):
             # Erlaube auch np.ndarray als innere Elemente, falls das vorkommt
            print(f"WARNUNG: Daten in {filename} sind keine Liste von Listen/np.ndarrays. Erstes Element Typ: {type(loaded_feature_vectors[0])}")
            return [], []

        data_for_class.extend(loaded_feature_vectors)
        target_for_class.extend([i] * len(loaded_feature_vectors))
        
    except FileNotFoundError:
        print(f"FEHLER: Datei {filename} nicht gefunden.")
        return [], []
    except pickle.UnpicklingError as e_pickle:
        print(f"FEHLER (pickle.UnpicklingError) beim Laden der konvertierten Datei {filename}: {e_pickle}")
        return [], [] 
    except Exception as e: 
        print(f"ALLGEMEINER FEHLER beim Laden der konvertierten Datei {filename}: {type(e).__name__} - {e}")
        return [], [] 
        
    return data_for_class, target_for_class

In [4]:
# In der Zelle, die mit "# load all the data" beginnt:
all_data = []
all_target = []

print(f"DEBUG: Klassen, die verarbeitet werden sollen: {classes}") 

for i in range(len(classes)):
    current_class_name = classes[i]
    # print(f"DEBUG: Verarbeite Klasse: {current_class_name} (Index {i})")
    
    data_i, target_i = load_pose_data(i) # Ruft die überarbeitete Funktion auf
    
    if data_i: # Nur hinzufügen, wenn Daten erfolgreich geladen und verarbeitet wurden
        # print(f"DEBUG: Für {current_class_name} wurden {len(data_i)} gültige Samples hinzugefügt.")
        all_data.extend(data_i)
        all_target.extend(target_i)
    else:
        print(f"WARNUNG: Keine Daten für Klasse {current_class_name} (Index {i}) geladen oder Daten waren ungültig.")

print(f"DEBUG: Gesamtzahl der gültig gesammelten Datenpunkte: {len(all_data)}")

if not all_data:
    print("FEHLER: Es konnten keine Daten erfolgreich aus den Pickle-Dateien geladen werden. `all_data` ist leer.")
    print("Bitte überprüfe die Fehlermeldungen oben für jede einzelne Datei.")
    # Du könntest hier das Notebook stoppen, da ohne Daten kein Training möglich ist.
    # raise RuntimeError("Keine Trainingsdaten geladen.")
else:
    try:
        # Konvertiere die Listen in numpy arrays, da scikit-learn diese erwartet
        all_data_np = np.array(all_data, dtype=float) # Explizit dtype=float
        all_target_np = np.array(all_target) # Targets sollten Integer sein
        
        print(f"Erfolgreich in NumPy-Arrays konvertiert. Shape all_data_np: {all_data_np.shape}, Shape all_target_np: {all_target_np.shape}")
        print(f"Gesamtzahl der Datenpunkte im NumPy-Array: {len(all_data_np)}")

        # Weise die NumPy-Arrays den ursprünglichen Variablennamen zu, wenn erfolgreich
        all_data = all_data_np
        all_target = all_target_np

    except ValueError as ve:
        print(f"FEHLER bei der Konvertierung der gesammelten Daten zu NumPy-Arrays: {ve}")
        print("Dies deutet darauf hin, dass trotz der Ladeversuche immer noch nicht-numerische Daten in `all_data` sind.")
        # Debugging-Schleife, um das problematische Element zu finden:
        for sample_idx, sample in enumerate(all_data):
            try:
                np.array(sample, dtype=float)
            except ValueError:
                print(f"Problem beim Konvertieren des Samples an Index {sample_idx} in all_data: {sample}")
                # Finde heraus, aus welcher Originaldatei dieses Sample kam (erfordert mehr Tracking, wenn mehrere Dateien erfolgreich waren)
                break
        # Hier könntest du entscheiden, das Notebook zu stoppen
        # raise ve
    except Exception as e_np:
        print(f"Ein anderer FEHLER bei der NumPy-Konvertierung: {e_np}")
        # raise e_np

# Stelle sicher, dass die Variablen für die nächste Zelle definiert sind, auch wenn sie leer sind,
# um Folgefehler durch nicht definierte Variablen zu vermeiden, falls das Laden komplett scheitert.
if 'all_data' not in locals() or not isinstance(all_data, np.ndarray) or all_data.size == 0:
    print("WARNUNG: `all_data` ist nach der Konvertierung leer oder kein NumPy-Array. Initialisiere als leeres Array.")
    all_data = np.empty((0,0)) # Dimensionen ggf. anpassen oder spezifischer Fehler
if 'all_target' not in locals() or not isinstance(all_target, np.ndarray) or all_target.size == 0:
    print("WARNUNG: `all_target` ist nach der Konvertierung leer oder kein NumPy-Array. Initialisiere als leeres Array.")
    all_target = np.empty((0,))

# Gib die finalen Längen aus
print(f"Länge von all_data (sollte NumPy sein): {len(all_data) if isinstance(all_data, np.ndarray) else 'Nicht NumPy oder leer'}")
print(f"Länge von all_target (sollte NumPy sein): {len(all_target) if isinstance(all_target, np.ndarray) else 'Nicht NumPy oder leer'}")

DEBUG: Klassen, die verarbeitet werden sollen: ['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']
DEBUG: Gesamtzahl der gültig gesammelten Datenpunkte: 222
Erfolgreich in NumPy-Arrays konvertiert. Shape all_data_np: (222, 10), Shape all_target_np: (222,)
Gesamtzahl der Datenpunkte im NumPy-Array: 222
Länge von all_data (sollte NumPy sein): 222
Länge von all_target (sollte NumPy sein): 222


In [5]:
# In der Zelle, die mit "# shuffule data" beginnt:

# Die print-Anweisungen ggf. an Python 3 anpassen, falls nicht schon geschehen
print(f"Länge von all_data vor Permutation: {len(all_data)}")
if len(all_data) == 0: # oder all_data.size == 0, wenn es sicher ein np.array ist
    print("WARNUNG: all_data ist leer. Training und Testsets werden ebenfalls leer sein.")
    # Setze Trainings- und Testdaten auf leere Arrays, um Fehler zu vermeiden
    # und um die Struktur für scikit-learn zu haben (shape (0, num_features))
    # Wir kennen die Anzahl der Features hier noch nicht sicher, wenn nichts geladen wurde.
    # Setze eine Standard-Feature-Anzahl, wenn du sie kennst (z.B. 10 laut Beschreibung)
    num_features_expected = 10 
    X_train = np.empty((0, num_features_expected))
    y_train = np.empty((0,))
    X_test = np.empty((0, num_features_expected))
    y_test = np.empty((0,))
    
    # Gebe aus, dass die Sets leer sind.
    print("X_train, y_train, X_test, y_test wurden als leere Arrays initialisiert.")

else:
    permutation = np.random.permutation(len(all_data))
    n_training_data = int(len(all_data) * 0.7)

    training_indices = permutation[:n_training_data]
    test_indices = permutation[n_training_data:]

    # Stelle sicher, dass all_data ein NumPy-Array ist für diese Art der Indizierung
    if not isinstance(all_data, np.ndarray):
        print("FEHLER: all_data ist keine NumPy-Array vor der Indizierung. Konvertierung fehlgeschlagen.")
        # Fallback, um TypeError zu vermeiden, aber das Training wird nicht funktionieren
        num_features_expected = 10 # Annahme
        X_train = np.empty((0, num_features_expected)) 
        y_train = np.empty((0,))
        X_test = np.empty((0, num_features_expected))
        y_test = np.empty((0,))
    else:
        X_train = all_data[training_indices]
        y_train = all_target[training_indices]

        X_test = all_data[test_indices]
        y_test = all_target[test_indices]
        print(f"Trainingsdaten erstellt: X_train shape {X_train.shape}, y_train shape {y_train.shape}")
        print(f"Testdaten erstellt: X_test shape {X_test.shape}, y_test shape {y_test.shape}")

Länge von all_data vor Permutation: 222
Trainingsdaten erstellt: X_train shape (155, 10), y_train shape (155,)
Testdaten erstellt: X_test shape (67, 10), y_test shape (67,)


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
clf.fit(X_train, y_train)

SVC(C=100.0, gamma=0.001)

### predicting

In [10]:
clf.predict(all_data[-1].reshape(1, -1)), all_target[-1]

(array([10]), 10)

In [11]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [12]:
expected = []
predicted = []

predicted_on_train = clf.predict(X_train)

# fülle die listen für die evaluierungsfunktion
expected.extend(list(y_train))       # die tatsächlichen trainings-labels
predicted.extend(list(predicted_on_train)) # die vorhergesagten labels für die trainingsdaten

evaluate(expected, predicted)


Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        10
           2       0.95      1.00      0.98        21
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00        16
           7       1.00      0.86      0.92         7
           8       1.00      1.00      1.00        17
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00        40

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0]
 [ 0  0 21  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [13]:
expected = []
predicted = []

predicted_on_test = clf.predict(X_test)

# fülle die listen für die evaluierungsfunktion
expected.extend(list(y_test))       # die tatsächlichen test-labels
predicted.extend(list(predicted_on_test)) # die vorhergesagten labels für die testdaten

evaluate(expected, predicted)


Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         5
          10       1.00      1.00      1.00        12

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [14]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [17]:
# Korrigierter Code für Zelle In[16]

# Lade den Klassifikator im Binärmodus
with open(ROBOT_POSE_CLF, 'rb') as f:
    clf2 = pickle.load(f)

# Wähle das letzte Sample aus all_data
single_sample = all_data[-1]

# Forme es in ein 2D-Array um (1 Zeile, n Spalten)
single_sample_reshaped = single_sample.reshape(1, -1)

# Mache die Vorhersage mit dem geladenen Klassifikator clf2
prediction_on_single_sample_with_clf2 = clf2.predict(single_sample_reshaped)

# Das tatsächliche Ziel für dieses Sample
actual_target_for_single_sample = all_target[-1]

# Gib die Vorhersage und das tatsächliche Ziel aus
print(f"Vorhersage mit geladenem clf2 für das letzte Sample: {prediction_on_single_sample_with_clf2}")
print(f"Tatsächliches Ziel für das letzte Sample: {actual_target_for_single_sample}")

# Optional: Um die Ausgabe der Zelle direkt als Tupel (Vorhersage, Ziel) zu haben, wie im Original-Notebook oft:
(prediction_on_single_sample_with_clf2, actual_target_for_single_sample)

Vorhersage mit geladenem clf2 für das letzte Sample: [10]
Tatsächliches Ziel für das letzte Sample: 10


(array([10]), 10)